<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="../image/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第三章: Attention

本章所需要的包

In [1]:
# 打印 PyTorch 版本，便于复现本章输出
from importlib.metadata import version

print("torch version:", version("torch"))


torch version: 2.10.0


- LLM的核心:Attention
- 译者:可以直接看论文呀!
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

<img src="../image/01.webp" width="500px">

<img src="../image/02.webp" width="600px">

## 3.1 长序列的建模

- 没有代码
- 逐字翻译文本通常不可行，因为源语言和目标语言在语法结构上存在差异：

<img src="../image/03.webp" width="400px">

- 在Transformer模型出现之前，机器翻译任务主要依赖于编码器(encoder)-解码器(decoder)架构的循环神经网络（RNNs）。
- 在这种架构中，编码器逐词处理源语言序列，并通过隐藏状态（神经网络中的中间层）生成输入序列的表示：

<img src="../image/04.webp" width="500px">

## 3.2 注意力机制高效捕获数据关系

- 本节不涉及代码。
- 借助注意力机制，文本生成解码器能够选择性地关注所有输入token，从而在生成特定输出token时，动态分配不同输入token的重要性系数

<img src="../image/05.webp" width="500px">

- Transformer中的自注意力机制是一种关键技术，它通过让序列中的每个位置与其他所有位置交互并计算相关性，从而增强输入表示的上下文信息。

<img src="../image/06.webp" width="300px">

## 3.3 自注意力关注的不同部分

### 3.3.1 无可变参数的自注意力模型

- 本节介绍了一种高度简化的自注意力变体，不包含任何可训练的权重。
- 该变体仅用于说明目的，并非Transformer中实际使用的注意力机制。
- 下一节（3.3.2节）将扩展此简易模型，实现真正的自注意力机制。
- 假设给定一个输入序列 $x^{(1)}$ 到 $x^{(T)}$：
  - 输入是一个文本（例如，一句已被处理为token嵌入的句子，如“Your journey starts with one step”），具体处理方法在第2章中已有描述。
  - 例如，$x^{(1)}$ 是表示单词“Your”的d维向量，以此类推。

- **目标：** 为输入序列中的每个元素 $x^{(i)}$（从 $x^{(1)}$ 到 $x^{(T)}$）计算上下文向量 $z^{(i)}$（$z$ 和 $x$ 的维度相同）。
    - 上下文向量 $z^{(i)}$ 是对输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权求和。
    - 上下文向量是针对特定输入的“上下文”相关表示。
      - 以第二个输入 $x^{(2)}$ 为例，说明具体计算过程。
      - 第二个上下文向量 $z^{(2)}$ 是对所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权求和，权重由相对于 $x^{(2)}$ 的注意力权重决定。
      - 注意力权重决定了每个输入元素对 $z^{(2)}$ 的贡献程度。
      - 简而言之，$z^{(2)}$ 是 $x^{(2)}$ 的增强版本，融合了与当前任务相关的所有其他输入元素的信息。

<img src="../image/07.webp" width="400px">

- （请注意，此图中的数字已截断至小数点后一位，以减少视觉干扰；其他图表中的数值也可能经过类似处理。）

- 按照惯例，未归一化的注意力值称为 **“注意力得分”**，而归一化后总和为1的注意力得分称为 **“注意力权重”**。

- 下方代码逐步演示了上图的操作过程

<br>

- **步骤 1：** 计算未归一化的注意力得分 $\omega$
- 假设使用第二个输入token作为查询，即 $q^{(2)} = x^{(2)}$，通过点积计算未归一化的注意力得分：
    - $\omega_{21} = x^{(1)} \cdot q^{(2)\top}$
    - $\omega_{22} = x^{(2)} \cdot q^{(2)\top}$
    - $\omega_{23} = x^{(3)} \cdot q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} \cdot q^{(2)\top}$
- 其中，$\omega$ 是希腊字母“欧米伽”，表示未归一化的注意力得分。
    - 在 $\omega_{21}$ 中，下标“21”表示以第2个元素为查询，与第1个元素计算得分。

- 假设我们有以下输入句子，该句子已根据第3章的描述嵌入到3维向量中（此处我们使用了一个非常小的嵌入维度进行说明，以便内容可以显示在页面上）： 

In [1]:
import torch

# 构造一个玩具输入序列 inputs：共有 6 个 token，每个 token 用 3 维向量表示
# inputs.shape == (num_tokens=6, d_in=3)
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
# 每一行对应一个 token 的输入向量 x^(i)，每一列对应一个特征维度


- （在本书中，我们遵循机器学习和深度学习的常见惯例：训练样本以行表示，特征值以列表示；对于上述张量，每一行表示一个词，每一列表示一个嵌入维度。）

- 本节的主要目标是演示如何以第二个输入序列 $x^{(2)}$ 作为查询，计算其上下文向量 $z^{(2)}$。

- 图中展示了该过程的第一步，即通过点积操作计算 $x^{(2)}$ 与所有其他输入元素之间的注意力得分 $\omega$。

<img src="../image/08.webp" width="400px">

- 我们以输入序列中的第2个元素 $x^{(2)}$ 为例，计算其上下文向量 $z^{(2)}$；稍后会将此方法推广至计算所有上下文向量。
- 第一步是通过计算查询 $x^{(2)}$ 与所有输入token的点积，得到未归一化的注意力得分：

In [2]:
# 步骤 1：计算未归一化注意力分数 omega
# 这里我们选择第 2 个 token（索引为 1）作为 query，意思是“以它为中心，关注其它单词”
query = inputs[1]  # 选取第2个 token（journey）作为查询序列（Query 向量）

# 创建一个空的 Tensor（与输入 token 数量相同），用于存放所有注意力分数 ω_{2,i}
# 这里 attn_scores_2 最终会存储 query（第2个 token）与每个输入 x^(i) 的点积（共 num_tokens 个）
attn_scores_2 = torch.empty(inputs.shape[0])  # shape: (num_tokens,)

# 针对每一个输入 token x^(i)，计算它与 query 的点积，作为未归一化的相关性分数
for i, x_i in enumerate(inputs):
    # torch.dot(x_i, query)：计算第 i 个 token 和 query 的向量点积，结果为一个标量
    # 这个标量反映了 x^(i) 与 query 的“相似程度”，值越大代表越相关
    attn_scores_2[i] = torch.dot(x_i, query)
    # 例如 i=0 时，是 x^(1) 与 x^(2) 做点积，对应 ω_{2,1}

# 打印所有注意力分数，结果为 shape=(6,) 的一维张量
# 这些分数就是“以第2个 token 作为 query”时，模型对其它每个 token 的初步关注度（未归一化）
print(attn_scores_2)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 补充说明：点积本质上是逐元素相乘并将所得积相加的一种简写表示：

In [3]:
# 补充：点积就是逐元素相乘再求和（这里示例计算 ω_21 = x^(1) · q^(2)）
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]
    # 累加 x^(1)[idx] * q^(2)[idx]

print(res)
print(torch.dot(inputs[0], query))
# 两种写法结果应一致


tensor(0.9544)
tensor(0.9544)


- **步骤 2：** 将未归一化的注意力得分（“欧米伽”，$\omega$）归一化，使其总和为1。
- 以下是一种简单的方法，用于将未归一化的注意力得分归一化：

<img src="../image/09.webp" width="500px">

In [4]:
# Step 2（简化版）：把分数除以总和，让权重之和为 1（注意：这不是 softmax）
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum() 

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实践中，使用softmax函数进行归一化更为常见，因为它能够更好地处理极端值，并且在训练过程中具有更理想的梯度特性，因此推荐使用。
- 下面是一个简单的softmax函数实现，用于缩放并对向量元素进行归一化，使它们的和为1：

In [5]:
# 一个“朴素版”的 softmax：仅用于演示 softmax 的基本原理（工程实践中建议用 torch.softmax）
def softmax_naive(x):
    # 1. 对输入的每个元素分别做指数运算
    #    这一步会把原始分数提升到非负、幅度更大的空间，拉大差异
    exp_x = torch.exp(x)
    # 2. 对所有指数后的结果求总和
    sum_exp_x = exp_x.sum(dim=0)
    # 3. 每个元素都除以总和，得到归一化后的“概率分布”
    softmax_x = exp_x / sum_exp_x
    return softmax_x

# 对注意力分数使用朴素 softmax 实现进行归一化
attn_weights_2_naive = softmax_naive(attn_scores_2)

# 输出归一化后的注意力权重
print("Attention weights:", attn_weights_2_naive)
# 输出权重之和，理想结果应为 1（概率分布的性质）
print("Sum:", attn_weights_2_naive.sum())
# softmax 的效果：将任意实数分数映射到 0~1 之间，且所有元素加起来等于 1
# 真正训练时建议直接用 torch.softmax（更高效且数值更安全）


Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上述简单实现可能会因输入值过大或过小而遭遇数值不稳定问题，导致溢出或下溢。
- 因此，在实践中，建议使用PyTorch内置的softmax函数，因为它经过高度优化，性能更佳：

In [ ]:
# ======== Step 2：使用 PyTorch 内置 softmax 对注意力分数进行归一化 ========
# softmax 会把原始的注意力分数（可以为任意实数，正负都行）缩放到[0,1]之间，
# 并且保证所有元素加起来等于 1（概率分布），这样能让每个分数都用来表示“相对重要性”。
# 这里用 torch.softmax 而不用自己实现的 softmax，可以保证数值稳定性（防止溢出、下溢），速度也更快。
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
# 解释一下 dim=0: 代表在第 0 维（一维向量的情况下就是对整个向量）做 softmax
# 如果 attn_scores_2 是更高维（如二维），则 dim 选择不同。

# 输出经过 softmax 归一化后的注意力权重
print("Attention weights:", attn_weights_2)
# 输出归一化后注意力权重的和，验证其是否等于 1（实际输出可能略有误差，但应非常接近 1）
print("Sum:", attn_weights_2.sum())


Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤 3**：通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘，并对结果向量求和，计算上下文向量 $z^{(2)}$：

<img src="../image/10.webp" width="500px">

In [ ]:
 # Step 3: 用注意力权重对输入向量加权求和，得到上下文向量 z^(2)

# 选定第2个输入token（这里的索引为1）作为query（即本轮"聚焦"的词）
query = inputs[1]  # inputs 是 shape 为 (6, 3) 的张量，表示 6 个 token 的嵌入向量

# 为该 query 初始化一个全零向量，用于累加加权求和的结果
# shape 设置为和单个 token 嵌入相同 (3,)
context_vec_2 = torch.zeros(query.shape)

# 遍历所有输入 token，逐个累加“注意力加权后”的 value 向量
# 通常情况下，inputs 既是 query 也是 value
for i, x_i in enumerate(inputs):
    # attn_weights_2[i]：第2个 token（作为query）对第i个 token 的注意力权重
    # x_i：第i个 token 的嵌入向量
    # attn_weights_2[i] * x_i：该向量在输出中的贡献（按权重缩放）
    context_vec_2 += attn_weights_2[i] * x_i
    # 累加所有加权后的向量（即加权求和）

# 输出该 query token 的上下文向量（self-attention 结果）
print(context_vec_2)


tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有token的attention score

#### 将其推广到所有输入序列标记：

- 上面，我们为输入2计算了注意力权重和上下文向量（如下图中高亮的行所示）。
- 接下来，我们将此计算推广到所有输入序列标记，计算对应的注意力权重和上下文向量。

<img src="../image/11.webp" width="400px">

- （请注意，图中的数字已四舍五入到小数点后两位；每一行的数值应相加为1.0或100%；其他图中的数字也进行了类似处理。）

- 在自注意力机制中，首先计算注意力得分，随后对这些得分进行归一化，得到总和为1的注意力权重。
- 接着，利用这些注意力权重对输入进行加权求和，生成上下文向量。

<img src="../image/12.webp" width="400px">

- 对所有成对元素应用之前的**步骤 1**，计算未归一化的注意力得分矩阵：

In [8]:
# 将 Step 1 推广到所有 token：计算所有成对元素的注意力得分矩阵 ω（shape: (T, T)）
attn_scores = torch.empty(6, 6)
#建立个空表来储存相关联程度

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
        # attn_scores[i, j] 表示第 i 个 token 对第 j 个 token 的相似度（未归一化分数）
print(attn_scores)
# 每一行对应一个 query token，每一列对应一个 key token


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 如果是矩阵相乘那么更有效率

In [9]:
# 上面的双重循环等价于一次矩阵乘法（更快、更简洁）
attn_scores = inputs @ inputs.T
print(attn_scores)
# (T, d_in) @ (d_in, T) -> (T, T)


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与**第二步**相似, 我们对每一行都要归一化操作:

In [10]:
# Step 2 推广到矩阵：对每一行做 softmax，使每个 query 的注意力权重和为 1
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)
# dim=-1 表示沿最后一维（列）归一化：每一行都是一个概率分布


tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 一个快速验证

In [11]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))
#验证一下大家加起来都是1

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 用**step 3** 计算所有的向量:

In [12]:
# Step 3 推广到所有 token：一次矩阵乘法得到所有上下文向量（shape: (T, d_in)）
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)
# (T, T) @ (T, d_in) -> (T, d_in)


tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


-  作为合理性检查，之前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上图的第二行找到：

In [13]:
# 与之前手工计算的 z^(2)（context_vec_2）应一致
print("Previous 2nd context vector:", context_vec_2)


Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 可调整参数的自注意力机制

- 以下的概念框架展示了本节中开发的自注意力机制,以及这种机制是如何融入本书和本章的整体叙述与结构。

<img src="../image/13.webp" width="400px">

### 3.4.1 手把手的计算attention的值

- 在本节中，我们实现了原始 Transformer 架构、GPT 模型以及大多数流行 LLM 中使用的自注意力机制。  
- 这种自注意力机制被称为“缩放点积注意力”（scaled dot-product attention）。  
- 整体思路与之前相似：  
  - 我们希望计算针对特定输入元素的上下文向量，即输入向量的加权和。  
  - 为此，我们需要生成注意力权重。  
- 如你所见，与之前介绍的基本注意力机制相比，只有一些细微差异：  
  - 最显著的区别是引入了在模型训练过程中更新的权重矩阵。  
  - 这些可训练的权重矩阵至关重要，它们使模型（尤其是注意力模块）能够学习生成“优质”的上下文向量。


<img src="../image/14.webp" width="600px">

- 按照步骤实现自注意力机制，我们将首先介绍三个训练权重矩阵 $W_q$、$W_k$ 和 $W_v$。  
- 这三个矩阵用于通过矩阵乘法将嵌入的输入标记 $x^{(i)}$ 映射到查询向量、键向量和值向量：
- (译者: 分别是Query、Key、Value,专有名词)   

  - 查询向量：$q^{(i)} = W_q \,x^{(i)}$  
  - 键向量：$k^{(i)} = W_k \,x^{(i)}$  
  - 值向量：$v^{(i)} = W_v \,x^{(i)}$  


- 输入 $x$ 和查询向量 $q$ 的嵌入维度可以相同，也可以不同，具体取决于模型的设计和实现方式。
- 在 GPT 模型中，输入和输出维度通常是相同的，但为了便于示范并更好地理解计算过程，这里我们选择了不同的输入和输出维度：

In [14]:
# 取出第 2 个 token 的输入向量 x^(2) 作为示例
x_2 = inputs[1] # second input element
# d_in: 输入向量维度（这里是 3）；d_out: Q/K/V 投影后的维度（这里设为 2，便于展示）
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2


- 下面，我们初始化三个权重矩阵；请注意，为了简化输出并便于示范，我们将 `requires_grad=False`
- 但如果我们要在模型训练中使用这些权重矩阵，应将 `requires_grad=True`，以便在训练过程中更新这些矩阵。

In [15]:
torch.manual_seed(123)
# 固定随机种子确保可复现性

# 三个可学习的投影矩阵：把输入 x 映射到 query/key/value 空间
# 形状都是 (d_in, d_out)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
# 注意：这里为了让输出更易读，requires_grad=False；真实训练中应为 True


- 计算这三个向量值

In [16]:
# 计算第 2 个 token 对应的 q^(2), k^(2), v^(2)（此处用矩阵乘法实现线性投影）
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value
# 结果向量维度为 d_out（这里是 2）
print(query_2)

tensor([0.4306, 1.4551])


- 我们可以清晰地看到,embedding被降维了:

In [17]:
# 对所有 token 做投影：得到 keys/values 矩阵
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)
# keys.shape == values.shape == (T, d_out)


keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步 **步骤 2** 中，我们通过计算查询向量和每个键向量之间的点积来计算未归一化的注意力得分：

<img src="../image/15.webp" width="600px">

In [18]:
# 计算一个单独的注意力得分：ω_22 = q^(2) · k^(2)
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 因为我们有六个输入,所以我们有六个attention score

In [19]:
# 一次性计算 q^(2) 对所有 key 的注意力得分：ω_2j（shape: (T,)）
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)
# (d_out,) @ (d_out, T) -> (T,)


tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="../image/16.webp" width="600px">

- 接下来，在 **步骤 3** 中，我们使用之前提到的 softmax 函数计算注意力权重（归一化后的注意力得分，总和为 1）。
- 与之前的不同之处在于，我们现在通过将注意力得分除以嵌入维度的平方根 $\sqrt{d_k}$（即 `d_k**0.5`）来对注意力得分进行缩放：

In [20]:
# 对注意力得分做缩放 softmax（scaled dot-product attention 的关键一步）
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
# 除以 sqrt(d_k) 可以避免 d_k 较大时 softmax 进入饱和区导致梯度不稳定
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="../image/17.webp" width="600px">

- 在**第四步**, 我们可以计算每一个token的向量了:

In [21]:
# 用注意力权重对 values 加权求和，得到上下文向量 z^(2)（shape: (d_out,)）
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)


tensor([0.3061, 0.8210])


### 3.4.2 自注意模块

- 下面是代码

In [22]:
import torch.nn as nn

# ============================================================================
# SelfAttention_v1: 最简自注意力层（单头）
# ============================================================================
# 功能说明：
#   实现标准的 scaled dot-product self-attention 机制
#   输入 x.shape == (T, d_in)，其中 T 是序列长度（token 数），d_in 是输入特征维度
#   输出 context_vec.shape == (T, d_out)，其中 d_out 是输出特征维度
#
# 核心思想：
#   每个 token 通过注意力机制"关注"序列中的所有 token（包括自己），
#   根据相关性加权聚合信息，生成融合了上下文的新表示
# ============================================================================
class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        """
        初始化自注意力层的可学习参数
        
        参数:
            d_in (int): 输入特征维度（例如词嵌入维度）
            d_out (int): 输出特征维度（Query/Key/Value 的投影维度）
        """
        super().__init__()
        
        # ====================================================================
        # 三个可学习权重矩阵：W_query, W_key, W_value
        # ====================================================================
        # 每个矩阵的形状均为 (d_in, d_out)
        # 作用：将输入 x 从 d_in 维空间线性投影到 d_out 维空间
        #
        # - W_query: 用于生成查询向量 Q，表示"当前 token 想要查询什么信息"
        # - W_key:   用于生成键向量 K，表示"每个 token 提供什么信息"
        # - W_value: 用于生成值向量 V，表示"每个 token 实际携带的信息内容"
        #
        # nn.Parameter 会将这些张量注册为模型的可训练参数
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        
        # 注意：这里使用 torch.rand 只是为了演示，会生成 [0, 1) 均匀分布的随机值
        # 实际训练时应使用更合适的初始化方案（如 Xavier/Kaiming 初始化）
        # 以避免梯度消失/爆炸问题

    def forward(self, x):
        """
        前向传播：计算自注意力输出
        
        参数:
            x (Tensor): 输入张量，shape == (T, d_in)
                       T 是序列长度，d_in 是输入特征维度
        
        返回:
            context_vec (Tensor): 上下文向量，shape == (T, d_out)
                                 每个 token 的输出都融合了序列中所有 token 的信息
        """
        
        # ====================================================================
        # 步骤 1: 线性投影 - 生成 Query, Key, Value
        # ====================================================================
        # 通过矩阵乘法将输入 x 投影到 Q/K/V 空间
        # x.shape: (T, d_in)  @  W.shape: (d_in, d_out)  =>  输出.shape: (T, d_out)
        
        keys = x @ self.W_key      # K = x * W_k, shape: (T, d_out)
        queries = x @ self.W_query # Q = x * W_q, shape: (T, d_out)
        values = x @ self.W_value  # V = x * W_v, shape: (T, d_out)
        
        # 解释：
        # - keys[i] 表示第 i 个 token 的"键"，用于被其他 token 查询
        # - queries[i] 表示第 i 个 token 的"查询"，用于查询其他 token
        # - values[i] 表示第 i 个 token 的"值"，是实际要聚合的信息
        
        # ====================================================================
        # 步骤 2: 计算注意力得分（未归一化）
        # ====================================================================
        # 通过 Q 和 K 的点积计算相似度矩阵
        # queries.shape: (T, d_out)  @  keys.T.shape: (d_out, T)  =>  attn_scores.shape: (T, T)
        
        attn_scores = queries @ keys.T  # 也称为 omega (Ω)
        
        # 解释 attn_scores[i, j] 的含义：
        # - 表示第 i 个 token（作为 query）对第 j 个 token（作为 key）的原始相关性得分
        # - 值越大表示第 i 个 token 越"关注"第 j 个 token
        # - 这是一个 (T, T) 的方阵，对角线元素表示 token 对自己的关注度
        
        # ====================================================================
        # 步骤 3: 缩放 + Softmax 归一化 -> 注意力权重
        # ====================================================================
        # 为什么要缩放（除以 sqrt(d_out)）？
        # - 当 d_out 较大时，点积 queries @ keys.T 的值会变得很大
        # - 过大的值会导致 softmax 进入饱和区（输出接近 0 或 1），梯度接近 0
        # - 缩放可以保持数值稳定，使梯度更健康，训练更稳定
        #
        # 为什么用 softmax？
        # - 将原始得分转换为概率分布（每行和为 1，所有值在 [0, 1] 之间）
        # - dim=-1 表示对最后一个维度（每一行）做 softmax
        # - 这样每个 query（每一行）的注意力权重总和为 1
        
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5,  # keys.shape[-1] 就是 d_out
            dim=-1
        )
        
        # attn_weights.shape: (T, T)
        # attn_weights[i, j] 表示第 i 个 token 对第 j 个 token 的归一化注意力权重
        # 满足：sum(attn_weights[i, :]) == 1.0（每个 query 的权重和为 1）
        
        # ====================================================================
        # 步骤 4: 加权聚合 Value -> 上下文向量
        # ====================================================================
        # 用注意力权重对 values 进行加权求和
        # attn_weights.shape: (T, T)  @  values.shape: (T, d_out)  =>  context_vec.shape: (T, d_out)
        
        context_vec = attn_weights @ values
        
        # 解释 context_vec[i] 的含义：
        # - 第 i 个 token 的输出表示
        # - 是序列中所有 token 的 value 向量的加权和
        # - 权重由 attn_weights[i, :] 决定（即第 i 个 token 对其他 token 的关注度）
        # - 因此 context_vec[i] 融合了整个序列的上下文信息
        
        return context_vec


# ============================================================================
# 测试代码：实例化并运行 SelfAttention_v1
# ============================================================================
torch.manual_seed(123)  # 固定随机种子，确保结果可复现
sa_v1 = SelfAttention_v1(d_in, d_out)  # 创建自注意力层实例
print(sa_v1(inputs))  # 前向传播，输出 shape: (T, d_out)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="../image/18.webp" width="400px">

- 我们可以使用 PyTorch 的 `Linear` 层简化上述实现，禁用偏置项后，`Linear` 层相当于矩阵乘法。
- 使用 `nn.Linear` 替代手动使用 `nn.Parameter(torch.rand(...))` 的一个主要优势是，`nn.Linear` 具有推荐的权重初始化方案，这有助于模型训练更加稳定。

In [23]:
# ============================================================================
# SelfAttention_v2: 用 nn.Linear 重写 v1（更符合 PyTorch 常见写法）
# ============================================================================
# 
# 与 SelfAttention_v1 的主要区别：
# - v1 使用 nn.Parameter(torch.rand(...)) 手动定义权重矩阵
# - v2 使用 nn.Linear 层，自带更好的权重初始化（例如 Kaiming/Xavier 初始化）
# - nn.Linear 的本质：y = xW^T + b（可选 bias）
#   当 bias=False 时，就是纯矩阵乘法，等价于 v1 的 x @ W_query
#
# 参数说明：
# - d_in: 输入嵌入维度（每个 token 的特征维度）
# - d_out: 输出维度（Q/K/V 的目标维度，通常称为 d_k 或 d_v）
# - qkv_bias: 是否在 Q/K/V 投影中使用偏置项（GPT 系列通常设为 False）

class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        
        # ====================================================================
        # 定义三个线性投影层：用于将输入 x 投影到 Query、Key、Value 空间
        # ====================================================================
        # nn.Linear(d_in, d_out, bias=qkv_bias) 的作用：
        # - 输入 shape: (batch_size, seq_len, d_in) 或 (seq_len, d_in)
        # - 输出 shape: (batch_size, seq_len, d_out) 或 (seq_len, d_out)
        # - 内部参数：权重矩阵 shape 为 (d_out, d_in)，偏置向量 shape 为 (d_out,)
        # - 计算公式：output = input @ weight.T + bias（如果 bias=True）
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)  # Query 投影层
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)  # Key 投影层
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)  # Value 投影层
        
        # 使用 nn.Linear 的优势：
        # 1. 自动使用推荐的权重初始化方案（例如 Kaiming uniform 初始化）
        # 2. 代码更简洁，不需要手动管理 nn.Parameter
        # 3. 与 PyTorch 生态系统更好地集成（例如自动注册参数、支持 .to(device) 等）

    def forward(self, x):
        """
        前向传播：计算自注意力的上下文向量
        
        参数：
            x: 输入张量，shape 为 (T, d_in)
               - T: 序列长度（token 数量）
               - d_in: 输入嵌入维度
        
        返回：
            context_vec: 上下文向量，shape 为 (T, d_out)
                        - 每个 token 的输出表示，融合了序列中所有 token 的信息
        """
        
        # ====================================================================
        # 步骤 1: 生成 Query、Key、Value
        # ====================================================================
        # 通过三个独立的线性投影层，将输入 x 投影到 Q/K/V 空间
        # 输入 x.shape: (T, d_in)
        # 输出 keys/queries/values.shape: (T, d_out)
        
        keys = self.W_key(x)      # Key 矩阵：用于被查询（与 Query 计算相似度）
        queries = self.W_query(x) # Query 矩阵：用于查询（主动寻找相关信息）
        values = self.W_value(x)  # Value 矩阵：用于聚合（实际被加权求和的内容）
        
        # ====================================================================
        # 步骤 2: 计算注意力得分（未归一化的相似度）
        # ====================================================================
        # 使用矩阵乘法计算 Query 和 Key 之间的点积相似度
        # queries.shape: (T, d_out)  @  keys.T.shape: (d_out, T)  =>  attn_scores.shape: (T, T)
        
        attn_scores = queries @ keys.T 
        
        # attn_scores[i, j] 的含义：
        # - 第 i 个 token 的 query 向量与第 j 个 token 的 key 向量的点积
        # - 值越大，表示第 i 个 token 对第 j 个 token 的"关注度"越高
        # - 此时还未归一化，只是原始的相似度分数
        
        # ====================================================================
        # 步骤 3: 缩放 + Softmax 归一化 -> 注意力权重
        # ====================================================================
        # 为什么要缩放（除以 sqrt(d_k)）？
        # - 当 d_out（即 d_k）较大时，点积的方差会变大，导致 softmax 的梯度变小
        # - 缩放可以稳定梯度，避免训练初期梯度消失
        # - 这是 Transformer 论文《Attention is All You Need》中的标准做法
        
        # keys.shape[-1] 就是 d_out（即 Key 的维度，也称为 d_k）
        # 缩放因子：1 / sqrt(d_k)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        
        # attn_weights.shape: (T, T)
        # attn_weights[i, j] 的含义：
        # - 第 i 个 token 对第 j 个 token 的归一化注意力权重
        # - 满足：sum(attn_weights[i, :]) == 1.0（每个 query 的权重和为 1）
        # - dim=-1 表示对最后一个维度（每一行）做 softmax
        
        # ====================================================================
        # 步骤 4: 加权聚合 Value -> 上下文向量
        # ====================================================================
        # 用注意力权重对 values 进行加权求和
        # attn_weights.shape: (T, T)  @  values.shape: (T, d_out)  =>  context_vec.shape: (T, d_out)
        
        context_vec = attn_weights @ values
        
        # context_vec[i] 的含义：
        # - 第 i 个 token 的输出表示
        # - 是序列中所有 token 的 value 向量的加权和
        # - 权重由 attn_weights[i, :] 决定（即第 i 个 token 对其他 token 的关注度）
        # - 因此 context_vec[i] 融合了整个序列的上下文信息
        
        # context_vec.shape == (T, d_out)
        return context_vec


# ============================================================================
# 测试代码：实例化并运行 SelfAttention_v2
# ============================================================================
# 固定随机种子，确保 nn.Linear 的权重初始化可复现
torch.manual_seed(789)

# 创建自注意力层实例
# - d_in: 输入维度（由前面的代码定义，通常是嵌入维度）
# - d_out: 输出维度（Q/K/V 的目标维度）
sa_v2 = SelfAttention_v2(d_in, d_out)

# 前向传播，输出 shape: (T, d_out)
# - inputs 是前面定义的输入张量，shape 为 (T, d_in)
# - 输出是每个 token 的上下文向量，融合了序列中所有 token 的信息
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- `SelfAttention_v1` 和 `SelfAttention_v2` 会给出不同的输出，因为它们使用了不同的初始权重矩阵。

## 3.5 对未出现的信息的隐藏

- 在casual attention，对角线以上的注意力权重被掩蔽，确保在计算上下文向量时，LLM 无法利用位置的信息来调整注意力权重。

<img src="../image/19.webp" width="400px">

### 3.5.1 因果自注意力机制

- 在这一节中，我们将把之前的自注意力机制转换为因果自注意力机制。
- 因果自注意力确保模型在预测序列中某个位置的值时，仅依赖于前面已知位置的输出，而不依赖于后续位置。
- 换句话说，这确保了每个下一个词的预测仅依赖于前面的词。
- 为了实现这一点，对于每个给定的标记，我们会将“未知的信息”（即输入文本中当前token之后的token）掩蔽掉：

<img src="../image/20.webp" width="600px">

- 为了说明和实现因果自注意力，让我们使用上一节中的注意力得分和权重：

In [25]:
# 为了演示因果 mask，这里复用上一节 sa_v2 的投影层来生成 Q/K
# 注意：这一步还没有应用 mask，只是在计算未掩蔽的注意力权重
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)
# attn_weights.shape == (T, T)，每一行是一个 query 的注意力分布


tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 隐藏未知信息的attention score最简单的方法是通过 PyTorch 的 `tril` 函数进行掩蔽，其中主对角线以下的元素（包括对角线本身）设置为 1，主对角线以上的元素设置为 0：

In [24]:
# 构造一个最简单的因果 mask：下三角为 1（允许关注自己及之前），上三角为 0（屏蔽未来）
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
# mask_simple.shape == (T, T)
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与这个mask相乘，以将对角线以上的注意力得分置为零：

In [25]:
# 如果在 softmax 之后再乘 mask，上三角会变成 0，但行和不再是 1（概率分布被破坏）
masked_simple = attn_weights*mask_simple
print(masked_simple)
# 下面会展示如何重新归一化，或更高效地在 softmax 之前掩蔽


tensor([[0.2098, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1385, 0.2379, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1390, 0.2369, 0.2326, 0.0000, 0.0000, 0.0000],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.0000, 0.0000],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.0000],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 如果在 softmax 之后进行掩蔽，它会破坏 softmax 所创建的概率分布。
- softmax 确保所有输出值的总和为 1。
- 如果在 softmax 之后进行掩蔽，就需要重新归一化输出，确保其总和为 1，这会使过程更加复杂，并可能带来意想不到的效果。

- 我们可以用以下方式确保所有的数据都是归一化的

In [26]:
# 重新归一化：让每一行的注意力权重和恢复为 1
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)
# masked_simple_norm 的每一行再次是概率分布


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3680, 0.6320, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2284, 0.3893, 0.3822, 0.0000, 0.0000, 0.0000],
        [0.2046, 0.2956, 0.2915, 0.2084, 0.0000, 0.0000],
        [0.1753, 0.2250, 0.2269, 0.1570, 0.2158, 0.0000],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 尽管我们在技术上已经完成了因果注意力机制的编码，但让我们简要地探讨一种更高效的方法，以实现与上述相同的效果。
- 因此，在注意力得分进入 softmax 函数之前，我们可以将对角线以上的未归一化注意力得分用负无穷大进行掩蔽，而不是将其置零并重新归一化：

<img src="../image/21.webp" width="450px">

In [29]:
# ============================================================================
# 因果注意力掩蔽（Causal Attention Masking）的高效实现
# ============================================================================
# 
# 目标：让每个 token 只能"看到"自己及之前的 token，看不到未来的 token
# 
# 方法：在 softmax 之前，将"未来位置"的注意力得分设为 -inf
#       这样 softmax(-inf) = 0，自动实现掩蔽且保持概率分布归一化
# 
# 优势：相比"先 softmax 再乘 0 再重新归一化"的方式，这种方法更高效且数值稳定
# ============================================================================

# 步骤 1：创建上三角掩码矩阵（Upper Triangular Mask）
# ----------------------------------------------------------------------------
# torch.triu() 返回一个上三角矩阵（包含对角线及以上的元素）
# diagonal=1 表示从主对角线上方第 1 条对角线开始保留（即主对角线本身为 0）
# 
# 示例（假设 context_length=6）：
#   mask = [[0, 1, 1, 1, 1, 1],    # 第 0 个 token 可以看到自己(0)，不能看到 1~5
#           [0, 0, 1, 1, 1, 1],    # 第 1 个 token 可以看到 0~1，不能看到 2~5
#           [0, 0, 0, 1, 1, 1],    # 第 2 个 token 可以看到 0~2，不能看到 3~5
#           [0, 0, 0, 0, 1, 1],    # ...
#           [0, 0, 0, 0, 0, 1],
#           [0, 0, 0, 0, 0, 0]]    # 第 5 个 token 可以看到所有 0~5
# 
# 其中 1 表示"需要被掩蔽的位置"（未来 token），0 表示"允许看到的位置"（当前及过去）
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
# mask.shape == (context_length, context_length)，即 (T, T)

# 步骤 2：将掩码应用到注意力得分矩阵
# ----------------------------------------------------------------------------
# attn_scores 的形状：(context_length, context_length)，即 (T, T)
# attn_scores[i, j] 表示第 i 个 query 对第 j 个 key 的原始注意力得分
# 
# masked_fill(condition, value)：
#   - 将 condition 为 True 的位置填充为 value
#   - mask.bool() 将 mask 转为布尔类型（1→True, 0→False）
#   - -torch.inf 表示负无穷大
# 
# 效果：
#   - 当 mask[i, j] == 1（即 j > i，未来位置）时，masked[i, j] = -inf
#   - 当 mask[i, j] == 0（即 j <= i，当前及过去位置）时，masked[i, j] 保持原值
# 
# 为什么用 -inf？
#   - softmax(x) = exp(x) / sum(exp(x))
#   - exp(-inf) = 0，所以 softmax(-inf) = 0
#   - 这样在后续 softmax 时，未来位置的注意力权重会自动变为 0
#   - 且由于 softmax 的归一化特性，剩余位置的权重和仍为 1（无需手动重新归一化）
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
# masked.shape == (context_length, context_length)
# masked[i, j] 在 j > i 的位置被置为 -inf，其余位置保持 attn_scores[i, j] 的原值

# 步骤 3：打印掩蔽后的注意力得分矩阵（用于调试和理解）
# ----------------------------------------------------------------------------
# 输出示例（假设 context_length=6）：
#   tensor([[ 0.2260,    -inf,    -inf,    -inf,    -inf,    -inf],
#           [ 0.3470,  0.5258,    -inf,    -inf,    -inf,    -inf],
#           [ 0.1520,  0.2340,  0.8920,    -inf,    -inf,    -inf],
#           [ 0.4210,  0.6780,  0.3450,  0.1230,    -inf,    -inf],
#           [ 0.7890,  0.2340,  0.5670,  0.4560,  0.9870,    -inf],
#           [ 0.3210,  0.8760,  0.4320,  0.6540,  0.2340,  0.7650]])
# 
# 观察：
#   - 第 0 行：只有第 0 列有值（只能看到自己）
#   - 第 1 行：第 0~1 列有值（可以看到 token 0 和 1）
#   - 第 5 行：所有列都有值（可以看到所有 token 0~5）
#   - 对角线以上全是 -inf（未来位置被完全掩蔽）
print(masked)

tensor([[0.9995,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.9544, 1.4950,   -inf,   -inf,   -inf,   -inf],
        [0.9422, 1.4754, 1.4570,   -inf,   -inf,   -inf],
        [0.4753, 0.8434, 0.8296, 0.4937,   -inf,   -inf],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654,   -inf],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


<img src="../image/07.webp" width="400px">

- （请注意，此图中的数字已截断至小数点后一位，以减少视觉干扰；其他图表中的数值也可能经过类似处理。）

- 结果显然是归一化的

In [30]:
# 在掩蔽后的得分上做 scaled softmax，得到真正的因果注意力权重
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4056, 0.5944, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2566, 0.3741, 0.3693, 0.0000, 0.0000, 0.0000],
        [0.2176, 0.2823, 0.2796, 0.2205, 0.0000, 0.0000],
        [0.1826, 0.2178, 0.2191, 0.1689, 0.2115, 0.0000],
        [0.1473, 0.2033, 0.1996, 0.1500, 0.1160, 0.1839]])


### 3.5.2 使用Dropout防止过拟合

- 此外，我们还应用了丢弃（Dropout）来减少训练过程中的过拟合。
- dropout可以应用于多个位置：
  - 例如，在计算注意力权重之后；
  - 或在将注意力权重与值向量相乘之后。
- 在这里，我们选择在计算注意力权重之后应用丢弃掩码，因为这种做法更为常见。

- 另外，在此示例中，我们使用了50%的丢弃率，这意味着随机屏蔽掉一半的注意力权重。（在后续训练GPT模型时，我们会使用更低的丢弃率，例如0.1或0.2。）

<img src="../image/22.webp" width="400px">

- 如果我们应用0.5（50%）的丢弃率，未被抛弃的值将相应地被缩放一个因子，1/0.5 = 2。
- 这种缩放通过公式 1 / (1 - `dropout_rate`) 计算得出。

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) 
# dropout rate=0.5：随机将一半元素置 0；未被置 0 的元素会乘以 1/(1-0.5)=2 进行缩放
example = torch.ones(6, 6) 
# 构造一个全 1 矩阵用来观察 dropout 的“置零 + 缩放”效果

print(dropout(example))
# 注意：dropout 只在训练模式下生效（nn.Module 默认是 train()）


tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [32]:
# 把 dropout 应用到注意力权重上：会随机屏蔽部分注意力连接（常见正则化手段）
torch.manual_seed(123)
print(dropout(attn_weights))


tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5132, 0.7482, 0.7386, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5646, 0.5592, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4357, 0.0000, 0.3378, 0.0000, 0.0000],
        [0.0000, 0.4065, 0.3991, 0.2999, 0.2320, 0.0000]])


- 生成的输出可能会因操作系统的不同而有所不同；
- 你可以在 [PyTorch 问题追踪器](https://github.com/pytorch/pytorch/issues/121595) 上了解更多内容。

### 3.5.3 实现一个简洁的因果自注意力

- 现在，我们准备实现一个完整的自注意力机制，包含因果掩码和dropout。
- 另一项任务是实现代码以处理包含多个输入的批次，确保我们的 `CausalAttention` 类能够支持第二章中实现的数据加载器所生成的批量输出。
- 为了简化起见，我们通过复制输入文本示例来模拟批量输入：

In [33]:
# ========== 模拟批量输入（Batch Input）==========
# 在实际训练中，我们通常会同时处理多个序列（batch），而不是单个序列
# 这里通过复制同一个 inputs 来模拟一个包含 2 个样本的 batch

# torch.stack 的作用：
# - 将多个相同形状的张量沿着新维度堆叠起来
# - dim=0 表示在第 0 维（最前面）新增一个维度作为 batch 维度
# - 例如：两个 (6, 3) 的张量 → stack 后变成 (2, 6, 3)

batch = torch.stack((inputs, inputs), dim=0)
# 此时 batch 的形状为：
# - batch_size = 2（两个样本）
# - num_tokens = 6（每个样本有 6 个 token）
# - d_in = 3（每个 token 的嵌入维度为 3）

print(batch.shape)  # 输出: torch.Size([2, 6, 3])
# 解读：2 个输入样本，每个样本 6 个 token，每个 token 的嵌入维度是 3

torch.Size([2, 6, 3])


- 缩放因子  \sqrt{d}  的引入解决了注意力机制中的数值不稳定问题。
- 它确保了即使嵌入维度  d  较大，点积得分也能被合理地控制在一个适当范围，方便 Softmax 生成平滑的注意力分布，且梯度不会过大或过小。

In [34]:
class CausalAttention(nn.Module):
    """
    因果自注意力（Causal Self-Attention）模块
    
    功能：
    - 实现单头的因果自注意力机制（单个注意力头）
    - 支持批量输入（batch processing）
    - 包含因果掩码（causal mask），确保每个位置只能关注它之前的位置（不能看到未来）
    - 包含 dropout 正则化，防止过拟合
    
    参数说明：
    - d_in: 输入嵌入维度（每个 token 的输入特征维度）
    - d_out: 输出嵌入维度（每个 token 的输出特征维度，也是 Q/K/V 的维度）
    - context_length: 上下文长度（支持的最大序列长度）
    - dropout: dropout 概率（训练时随机丢弃注意力权重的比例）
    - qkv_bias: 是否在 Q/K/V 的线性变换中使用偏置项（默认 False）
    
    输入输出：
    - 输入 x: shape = (batch_size, num_tokens, d_in)
    - 输出 context_vec: shape = (batch_size, num_tokens, d_out)
    """

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        
        # 保存输出维度，后续用于缩放点积注意力分数
        self.d_out = d_out
        
        # ========== 定义 Q/K/V 的线性变换层 ==========
        # 这三个线性层将输入 x 分别映射为 Query、Key、Value
        # 每个线性层的作用：x (d_in) -> Q/K/V (d_out)
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)  # Query 投影矩阵
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)  # Key 投影矩阵
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)  # Value 投影矩阵
        
        # ========== Dropout 层 ==========
        # 用于在训练时随机丢弃部分注意力权重，防止过拟合
        # 注意：只在训练时生效（model.train()），验证/测试时自动关闭（model.eval()）
        self.dropout = nn.Dropout(dropout)
        
        # ========== 因果掩码（Causal Mask）==========
        # register_buffer: 将 mask 注册为模型的缓冲区（buffer）
        # - 不是可训练参数（不会被优化器更新）
        # - 会随模型一起保存和加载
        # - 会自动跟随模型移动到相应设备（CPU/GPU）
        
        # torch.triu: 生成上三角矩阵（upper triangular）
        # - torch.ones(context_length, context_length): 创建全 1 方阵
        # - diagonal=1: 主对角线上方（不包括主对角线）的元素保留为 1，其余为 0
        # 
        # 示例（context_length=4）：
        # [[0, 1, 1, 1],     <- 第 0 个 token 不能看到位置 1,2,3（未来）
        #  [0, 0, 1, 1],     <- 第 1 个 token 不能看到位置 2,3（未来）
        #  [0, 0, 0, 1],     <- 第 2 个 token 不能看到位置 3（未来）
        #  [0, 0, 0, 0]]     <- 第 3 个 token 可以看到所有位置（包括自己）
        # 
        # 其中 1 表示"需要被掩盖的位置"（未来位置），0 表示"可以关注的位置"（当前及过去）
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        """
        前向传播
        
        参数：
        - x: 输入张量，shape = (batch_size, num_tokens, d_in)
        
        返回：
        - context_vec: 上下文向量，shape = (batch_size, num_tokens, d_out)
        """
        
        # ========== 1. 获取输入维度 ==========
        b, num_tokens, d_in = x.shape
        # b: batch_size（批次大小，一次处理多少个序列）
        # num_tokens: 序列长度 T（当前输入的 token 数量）
        # d_in: 输入嵌入维度（每个 token 的特征维度）
        
        # ========== 2. 计算 Q/K/V ==========
        # 通过线性变换将输入 x 映射为 Query、Key、Value
        keys = self.W_key(x)      # shape: (b, num_tokens, d_out)
        queries = self.W_query(x) # shape: (b, num_tokens, d_out)
        values = self.W_value(x)  # shape: (b, num_tokens, d_out)
        
        # ========== 3. 计算注意力分数（Attention Scores）==========
        # 使用 Query 和 Key 的点积计算相似度
        # queries @ keys.transpose(1, 2):
        # - queries: (b, num_tokens, d_out)
        # - keys.transpose(1, 2): (b, d_out, num_tokens) <- 交换第 1 和第 2 维
        # - 结果: (b, num_tokens, num_tokens)
        # 
        # 含义：attn_scores[i, j] 表示第 i 个 token 对第 j 个 token 的"原始关注度"
        attn_scores = queries @ keys.transpose(1, 2)
        
        # ========== 4. 应用因果掩码（Causal Mask）==========
        # masked_fill_: 原地操作（in-place），将满足条件的位置填充为指定值
        # - self.mask.bool(): 将 mask 转为布尔类型（1 -> True, 0 -> False）
        # - [:num_tokens, :num_tokens]: 切片，只取前 num_tokens 行和列
        #   （因为实际输入的序列长度可能小于 context_length）
        # - -torch.inf: 负无穷大
        # 
        # 作用：将"未来位置"（mask 中为 1 的位置）的注意力分数设为 -∞
        # 这样在后续 softmax 时，这些位置的权重会变为 0，实现"不能看到未来"的效果
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], 
            -torch.inf
        )
        
        # ========== 5. 计算注意力权重（Attention Weights）==========
        # 使用 softmax 将注意力分数归一化为概率分布
        # 
        # 缩放因子 keys.shape[-1]**0.5（即 √d_out）的作用：
        # - 当 d_out 较大时，Q·K 的点积值会变得很大
        # - 除以 √d_out 可以将分数缩放到合理范围，避免 softmax 饱和
        # - 饱和问题：如果输入 softmax 的值过大，梯度会接近 0，导致训练困难
        # 
        # dim=-1: 对最后一维（每个 query 对应的所有 key）做 softmax
        # 结果：每一行的权重和为 1，表示该 token 对所有位置的注意力分布
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5,  # 缩放因子 √d_out
            dim=-1
        )
        
        # ========== 6. 应用 Dropout ==========
        # 在训练时随机丢弃部分注意力权重，防止过拟合
        # 在验证/测试时，dropout 会自动关闭（不丢弃任何权重）
        attn_weights = self.dropout(attn_weights)
        
        # ========== 7. 计算上下文向量（Context Vector）==========
        # 使用注意力权重对 Value 进行加权求和
        # attn_weights @ values:
        # - attn_weights: (b, num_tokens, num_tokens)
        # - values: (b, num_tokens, d_out)
        # - 结果: (b, num_tokens, d_out)
        # 
        # 含义：context_vec[i] 是第 i 个 token 的上下文表示
        # 它是所有可见位置（当前及过去）的 value 的加权和
        context_vec = attn_weights @ values
        
        return context_vec


# ========== 测试代码 ==========
# 设置随机种子，确保结果可复现
torch.manual_seed(123)

# 获取上下文长度（序列长度）
context_length = batch.shape[1]  # batch.shape = (2, 6, 3)，所以 context_length = 6

# 创建 CausalAttention 实例
# - d_in: 输入维度（从 batch 推断）
# - d_out: 输出维度
# - context_length: 支持的最大序列长度
# - 0.0: dropout 概率为 0（不使用 dropout，便于调试）
ca = CausalAttention(d_in, d_out, context_length, 0.0)

# 前向传播：计算上下文向量
context_vecs = ca(batch)

# 打印结果
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)
# 预期输出 shape: (2, 6, d_out)
# - 2: batch_size（2 个样本）
# - 6: num_tokens（每个样本 6 个 token）
# - d_out: 输出维度（每个 token 的上下文表示维度）

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Dropout仅在训练时要使用,验证时不需要

<img src="../image/23.webp" width="500px">

## 3.6 拓展单头至多方注意

### 3.6.1 堆叠多个单头注意力层

- 以下是之前实现的自注意力机制总结（为简化起见，未展示因果和dropout掩码）：

- 这种机制也称为单头注意力：

<img src="../image/24.webp" width="400px">

- 我们通过堆叠多个单头注意力模块来构建多头注意力模块：

<img src="../image/25.webp" width="400px">

- 多头注意力的核心思想是使用不同的学习到的线性投影，并行地多次运行注意力机制。这使得模型能够在不同位置同时关注来自不同表示子空间的信息。

在 Python 中，super().__init__() 是一种调用父类（基类）构造函数的方法，常用于类继承的场景中。它确保子类能够正确初始化父类的属性和方法。

In [35]:
# MultiHeadAttentionWrapper: 用多个 CausalAttention 头并行计算，然后在最后一维拼接
# 注意：这是"直观实现"，便于理解；下一节会实现更紧凑高效的 MultiHeadAttention
class MultiHeadAttentionWrapper(nn.Module):
    """
    多头注意力的包装器实现（直观版本）
    
    核心思想：
    - 创建多个独立的 CausalAttention 注意力头
    - 每个头独立处理输入，学习不同的表示子空间
    - 最后将所有头的输出在特征维度上拼接
    
    这种实现方式易于理解，但不如下一节的 MultiHeadAttention 高效
    """

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        """
        初始化多头注意力包装器
        
        参数:
            d_in: 输入特征维度（每个 token 的嵌入维度）
            d_out: 每个注意力头的输出维度
            context_length: 上下文长度（序列中的 token 数量）
            dropout: dropout 概率
            num_heads: 注意力头的数量
            qkv_bias: 是否在 Q/K/V 线性层中使用偏置
        
        注意：
            - 最终输出维度将是 num_heads * d_out
            - 例如：num_heads=2, d_out=2 → 最终输出维度=4
        """
        super().__init__() 
        
        # 创建 num_heads 个独立的因果注意力头
        # 每个头都是一个完整的 CausalAttention 实例，有自己的 Q/K/V 权重矩阵
        # nn.ModuleList 确保这些子模块被正确注册，参数会被优化器追踪
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        """
        前向传播：并行计算所有注意力头，然后拼接结果
        
        参数:
            x: 输入张量，shape = (batch_size, num_tokens, d_in)
        
        返回:
            拼接后的上下文向量，shape = (batch_size, num_tokens, num_heads * d_out)
        
        工作流程:
            1. 每个注意力头独立处理输入 x
            2. 每个头输出 shape = (batch_size, num_tokens, d_out)
            3. 在最后一维（特征维度）拼接所有头的输出
            4. 最终输出 shape = (batch_size, num_tokens, num_heads * d_out)
        """
        # 对每个头调用 forward，得到一个列表，每个元素 shape = (batch_size, num_tokens, d_out)
        # torch.cat(..., dim=-1) 在最后一维拼接，得到 (batch_size, num_tokens, num_heads * d_out)
        return torch.cat([head(x) for head in self.heads], dim=-1)
        # 拼接后输出维度为 num_heads * d_out


# ============ 测试多头注意力包装器 ============

# 设置随机种子以确保结果可复现
torch.manual_seed(123)

# 从 batch 中获取上下文长度（序列中的 token 数量）
context_length = batch.shape[1]  # batch.shape = (batch_size, num_tokens, d_in)

# 定义输入和输出维度
d_in, d_out = 3, 2
# d_in=3: 输入嵌入维度（与 batch 的最后一维匹配）
# d_out=2: 每个注意力头的输出维度

# 创建多头注意力实例
# num_heads=2: 使用 2 个注意力头
# dropout=0.0: 测试时不使用 dropout
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

# 前向传播：计算上下文向量
context_vecs = mha(batch)
# 输入 batch shape: (batch_size, num_tokens, 3)
# 输出 context_vecs shape: (batch_size, num_tokens, 4)
#   - 4 = num_heads * d_out = 2 * 2

# 打印结果
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)
# 预期输出 shape: (batch_size, num_tokens, 4)
# 例如：如果 batch_size=2, num_tokens=6，则输出 shape=(2, 6, 4)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上述实现中，嵌入维度为4，因为我们将 `d_out=2` 作为键、查询和值向量以及上下文向量的嵌入维度。由于使用了2个注意力头，输出嵌入维度为 2 * 2 = 4。

### 3.6.2 利用权重拆分实现多头注意力

- 尽管上述实现是一种直观且功能完整的多头注意力机制（通过封装之前的单头注意力 `CausalAttention` 实现），我们仍可以编写一个独立的 `MultiHeadAttention` 类来实现相同的功能。

- 在这个独立的 `MultiHeadAttention` 类中，我们不会将单个注意力头进行拼接。
- 相反，我们会创建独立的 W_query、W_key 和 W_value 权重矩阵，并将它们拆分为每个注意力头的单独矩阵：

In [36]:
# MultiHeadAttention: 将所有头的 Q/K/V 一次性算出，再 reshape/transpose 拆成多头并行计算
class MultiHeadAttention(nn.Module):
    """
    多头注意力机制的独立实现
    
    与 MultiHeadAttentionWrapper 不同，这个类不是通过拼接多个单头注意力实现的，
    而是通过权重拆分的方式：先计算完整的 Q/K/V，再将它们拆分成多个头并行计算。
    
    参数:
        d_in: 输入嵌入维度
        d_out: 输出嵌入维度（必须能被 num_heads 整除）
        context_length: 上下文长度（序列最大长度）
        dropout: dropout 概率
        num_heads: 注意力头的数量
        qkv_bias: 是否在 Q/K/V 线性层中使用偏置
    """
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        # 确保输出维度能被头数整除，这样每个头才能有相同的维度
        assert (d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"
        # 确保每个头的维度 head_dim 是整数
            
        # 保存关键参数
        self.d_out = d_out  # 总输出维度
        self.num_heads = num_heads  # 注意力头的数量
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim
        # head_dim: 每个注意力头的通道数；d_out = num_heads * head_dim
        # 例如：d_out=4, num_heads=2 => head_dim=2
        
        # 创建 Q/K/V 的线性变换层
        # 注意：这里的输出维度是 d_out（而不是 head_dim），稍后会拆分成多个头
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        # 输出投影层：将多头拼接后的结果再做一次线性变换
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        # out_proj: 将拼接后的多头输出再做一次线性变换（常见做法）
        # 这是 Transformer 标准实现的一部分，虽然不是严格必需的
        
        # Dropout 层，用于正则化
        self.dropout = nn.Dropout(dropout)
        
        # 注册因果掩码（上三角矩阵）作为缓冲区
        # register_buffer: 将 mask 注册为模型的一部分，但不作为可训练参数
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
        # 上三角掩码，确保因果性（每个 token 只能看到它之前的 token）
        # diagonal=1: 主对角线上方的元素为 1，其余为 0
        # 例如 3x3 的掩码：
        # [[0, 1, 1],
        #  [0, 0, 1],
        #  [0, 0, 0]]

    def forward(self, x):
        """
        前向传播
        
        参数:
            x: 输入张量，shape = (batch_size, num_tokens, d_in)
        
        返回:
            context_vec: 上下文向量，shape = (batch_size, num_tokens, d_out)
        """
        # 获取输入的形状信息
        b, num_tokens, d_in = x.shape
        # b: batch_size（批次大小）
        # num_tokens: 序列长度（当前批次中的 token 数量）
        # d_in: 输入嵌入维度

        # 步骤 1: 计算 Q/K/V（对整个 d_out 维度）
        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)  # Shape: (b, num_tokens, d_out)
        values = self.W_value(x)  # Shape: (b, num_tokens, d_out)
        # 此时 Q/K/V 的最后一维是完整的 d_out，还没有拆分成多个头
        
        # 步骤 2: 将 Q/K/V 拆分成多个头
        # 下面把最后一维 d_out 拆成 (num_heads, head_dim)，让每个头独立做 attention
        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        # view 操作将 d_out 维度重塑为 (num_heads, head_dim)
        # 例如：(2, 6, 4) -> (2, 6, 2, 2)，其中 4 = 2 * 2
        
        # 步骤 3: 调整维度顺序，将 num_heads 提前
        # 把 num_heads 维度提前：便于并行计算每个 head 的注意力
        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        # transpose(1, 2): 交换第 1 和第 2 维
        # 这样做的好处：每个头的计算可以并行进行，提高效率
        
        # 步骤 4: 计算注意力分数（缩放点积注意力）
        # 计算缩放点积注意力
        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head
        # queries: (b, num_heads, num_tokens, head_dim)
        # keys.transpose(2, 3): (b, num_heads, head_dim, num_tokens)
        # 结果 attn_scores: (b, num_heads, num_tokens, num_tokens)
        # 对于每个头，计算所有 token 对之间的相似度
        
        # 步骤 5: 应用因果掩码
        # 将掩码缩减到当前 token 数量，并转换为布尔型
        # 进而实现动态遮蔽,所以不用另开好几个数组
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # 只取掩码的前 num_tokens 行和列（因为实际序列长度可能小于 context_length）
        # bool(): 将 0/1 转换为 False/True
        
        # 遮蔽矩阵
        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        # masked_fill_: 原地操作，将 mask_bool 为 True 的位置填充为 -inf
        # -inf 在 softmax 后会变成 0，实现因果性（未来 token 的注意力权重为 0）
        
        # 步骤 6: 归一化（Softmax）并应用 Dropout
        #归一化
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        # 缩放因子：keys.shape[-1]**0.5 = sqrt(head_dim)
        # 缩放的目的：防止点积结果过大，导致 softmax 梯度消失
        # dim=-1: 在最后一维（num_tokens）上做 softmax，使每行和为 1
        attn_weights = self.dropout(attn_weights)
        # 对注意力权重应用 dropout，用于正则化

        # 步骤 7: 计算上下文向量（加权求和）
        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        # attn_weights: (b, num_heads, num_tokens, num_tokens)
        # values: (b, num_heads, num_tokens, head_dim)
        # 结果: (b, num_heads, num_tokens, head_dim)
        # transpose(1, 2): 将维度调整回 (b, num_tokens, num_heads, head_dim)
        # 把 head 维度再换回到 (b, num_tokens, num_heads, head_dim)
        
        # 步骤 8: 合并多个头的输出
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        #对上下文向量的形状进行调整，确保输出的形状
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        # contiguous(): 确保内存连续，便于 view 正确工作
        # view: 将 (b, num_tokens, num_heads, head_dim) 重塑为 (b, num_tokens, d_out)
        # 这相当于将所有头的输出拼接起来
        
        # 步骤 9: 应用输出投影层
        context_vec = self.out_proj(context_vec) # optional projection
        # 最后的线性变换，将拼接后的多头输出再做一次变换
        # 这是 Transformer 的标准做法，有助于模型学习更复杂的表示

        return context_vec

# ============ 测试代码 ============

# 设置随机种子以确保可重复性
torch.manual_seed(123)

# 获取输入数据的形状信息
batch_size, context_length, d_in = batch.shape
# batch 是之前定义的输入数据，shape = (batch_size, num_tokens, d_in)

# 定义输出维度
d_out = 2
# 注意：这里 d_out=2，num_heads=2，所以每个头的维度 head_dim = 2/2 = 1

# 创建多头注意力实例
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
# d_in: 输入维度（与 batch 的最后一维匹配）
# d_out=2: 输出维度
# context_length: 上下文长度
# 0.0: dropout 概率（测试时设为 0）
# num_heads=2: 使用 2 个注意力头

# 前向传播：计算上下文向量
context_vecs = mha(batch)
# 输入 batch shape: (batch_size, num_tokens, d_in)
# 输出 context_vecs shape: (batch_size, num_tokens, d_out)

# 打印结果
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)
# 预期输出 shape: (batch_size, num_tokens, d_out)
# 例如：如果 batch_size=2, num_tokens=6, d_out=2，则输出 shape=(2, 6, 2)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，上述实现本质上是 `MultiHeadAttentionWrapper` 的重写版本，并且更加高效。  
- 生成的输出看起来略有不同，因为随机权重初始化有所不同，但两者都是完全可用的实现，可以在我们将在后续章节中实现的 GPT 类中使用。  
- 另外，值得注意的是，我们在上面的 `MultiHeadAttention` 类中添加了一个线性投影层（`self.out_proj`）。这只是一个线性变换，不改变维度。在 LLM 实现中，使用这样的投影层是标准做法，但它并非严格必要（近期的研究表明，去除该层不会影响模型的表现；请参阅本章末尾的进一步阅读部分）。  


<img src="../image/26.webp" width="400px">

- 请注意，如果你对上述内容的紧凑和高效实现感兴趣，可以考虑使用PyTorch中的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类。

- 由于上述实现初看起来可能有些复杂，我们来看一下执行 `attn_scores = queries @ keys.transpose(2, 3)` 时会发生什么：

In [37]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))
# 对每个 head 单独计算相关性矩阵：(num_tokens, head_dim) @ (head_dim, num_tokens)
# 输出 shape 为 (b, num_heads, num_tokens, num_tokens)


tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现将处理四维输入张量，使得矩阵乘法在最后两个维度（`num_tokens`, `head_dim`）之间进行，然后对各个头进行重复计算。  

- 例如，以下方法提供了一种更紧凑的方式来分别计算每个头的矩阵乘法：  


In [38]:
first_head = a[0, 0, :, :]
# 取出第一个 head：shape (num_tokens, head_dim)
first_res = first_head @ first_head.T
# 得到该 head 的相关性矩阵：shape (num_tokens, num_tokens)
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 总结与收获

- 请参阅 [./multihead-attention.ipynb](./multihead-attention.ipynb) 代码笔记本，它是数据加载器（第2章）的简洁版本，加上我们在本章实现的多头注意力类，后续章节中训练GPT模型时将需要使用。
- 你可以在 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 中找到习题解答。